<a href="https://colab.research.google.com/github/Viktory4121/MORD/blob/main/%D0%94%D0%9E%D0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Динамические окрестностные модели (ДОМ)
####Вариант 5

In [113]:
import numpy as np
import pandas as pd
import random
import copy
import math
from prettytable import PrettyTable
import itertools as it

In [114]:
#A={a1, a2, a3}

#какие дуги входят в позицию:
#Ox[a1]={a1}
#Ox[a2]={}
#Ox[a3]={a1, a2}

#в какие позиции входят v
#Ov[a1]={a1}
#Ov[a2]={a2}
#Ov[a3]={a3}

#количества состояний/входов/выходов
elem = 100
count = [3, 3, 3]
Sx = np.array([[1, 0, 1], [0, 0, 1], [0, 0, 0]])
Sv = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

print("Матрица смежности:")
print("Sx = ")
print(Sx)
print("Sv = ")
print(Sv)

Матрица смежности:
Sx = 
[[1 0 1]
 [0 0 1]
 [0 0 0]]
Sv = 
[[1 0 0]
 [0 1 0]
 [0 0 1]]


In [115]:
def x1(x, v):
  return math.sin(math.log(abs(v[0])) / x[0])

def x2(x, v):
  return math.cos(v[1]**3)

def x3(x, v):
  return math.sin(math.tan(x[0] / x[2]) - 3**(v[2]))

def v1(x, v):
  return (v[0] + math.tan(x[0])**2)

def v2(x, v):
  return (0.2 * v[1])

def v3(x, v):
  return (math.exp(x[0]) + .45 * math.log2(abs(x[1] * v[2])))

def y1(x, v):
  return (x[0] / v[0]) * math.sqrt(abs(x[0]))

def y2(x, v):
  return math.sqrt(abs(math.sin(v[1])))

def y3(x, v):
  return v[2] * x[0] / (x[2] + 0.1)

functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]

In [116]:
#Метод псевдообращения Бена-Израиля
def BenIsrael(A, eps = 0.000001): #А - np.array, eps - точность вычислительная
  I = np.diag(np.diag(np.ones([len(A),len(A)])))
  buf = 1
  i = 1

  if(np.linalg.norm(A, ord='fro') != 0): #если не нулевая норма
    gamma = 1.6 / (np.linalg.norm(A, ord='fro')**2)
    A1 = gamma * (A.T)
    i += 1
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
  else: #если нулевая норма
    A1 = np.zeros((len(A[0]), len(A)))
    buf = 0

  while(E >= eps and buf):
    A1 = A2.copy()
    A2 = A1.dot(2*I - A.dot(A1))
    E = np.linalg.norm(A2 - A1, ord=np.inf) / np.linalg.norm(A1, ord=np.inf)
    i += 1

  return A1

#решение СЛАУ вида AX=B
def solvingSLAE(A, B):
  A = np.array(A.copy())

  #проверка на бесконечное количество решений
  A_psev = BenIsrael(A)

  X = A_psev.dot(B)
  return X

#=================================================================
#Генерация случайного вектора из 3-х элементов в диапазоне от -3 до 3
def generationRandomVector(elem = 3, start = -3, end = 3):
  vector = []
  for i in range(elem):
    vector.append(random.uniform(start, end))
  return vector

#=================================================================
#создание столбца таблицы
def createVariants(function, x, v, typeFunc = 1, elem = 100):
  vector = []

  if(typeFunc == 0):
    for i in range(elem):
      vector.append(function(generationRandomVector(), generationRandomVector()))
  else:
    for i in range(elem):
      vector.append(function(x[i], v[i]))

  return vector

#=================================================================
#Создание ДОМ в определённый момент времени
def createTableDNM(functions, count, step, oldTable = []): #count(x, v, y)
  table = []
  vector = []
  columns = createColumns(step, count)

  if(step == 0):
    for i in range(sum(count[0:2])):
      table.append(createVariants(functions[i], [], [], 0)) #xi, vi

  else:
    for i in range(count[0]): #xi
      table.append(createVariants(functions[i], (np.array(oldTable))[:, 0:count[0]], (np.array(oldTable))[:, count[0]:sum(count[0:2])]))
    for i in range(count[1]): #vi
      table.append(createVariants(functions[count[0] + i], [], [], 0))

  for i in range(count[2]): #yi
    table.append(createVariants(functions[sum(count[0:2]) + i], (np.array(table).T)[:, 0:count[0]], (np.array(table).T)[:, count[0]:sum(count[0:2])]))

  table = pd.DataFrame(np.array(table).T, columns = columns)
  return table

#заполнение
def createColumns(step, count):
  columns = []
  for i in range(count[0]):
    columns.append("x" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[1]):
    columns.append("v" + str(i + 1) + "(" + str(step) + ")")
  for i in range(count[2]):
    columns.append("y" + str(i + 1) + "(" + str(step) + ")")
  return columns

In [117]:
#functions = [x1, x2, x3, v1, v2, v3, y1, y2, y3]
model_0 = createTableDNM(functions, count, 0)
model_1 = createTableDNM(functions, count, 1, np.array(model_0))
model_2 = createTableDNM(functions, count, 2, np.array(model_1))
model_3 = createTableDNM(functions, count, 3, np.array(model_2))
model_4 = createTableDNM(functions, count, 4, np.array(model_3))
model_5 = createTableDNM(functions, count, 5, np.array(model_4))
model_6 = createTableDNM(functions, count, 6, np.array(model_5))
model_7 = createTableDNM(functions, count, 7, np.array(model_6))
model_8 = createTableDNM(functions, count, 8, np.array(model_7))
model_9 = createTableDNM(functions, count, 9, np.array(model_8))
model_10 = createTableDNM(functions, count, 10, np.array(model_9))

In [118]:
model_9

,x1(9),x2(9),x3(9),v1(9),v2(9),v3(9),y1(9),y2(9),y3(9)
0,-0.256393,0.986387,-0.594927,-0.124506,-0.498841,10.619837,1.042722e+00,0.691670,5.501528
1,0.377749,0.999324,0.974466,0.647070,-0.293036,11.915311,3.588017e-01,0.537457,4.189059
2,0.893641,0.999932,-0.457796,-0.460634,-0.131483,20.778472,-1.833957e+00,0.362083,-51.896833
3,0.990310,0.999925,-0.331439,13.884109,0.054866,15.828629,7.098042e-02,0.234177,-67.729553
4,0.999898,0.999999,0.975114,-1.504231,0.512417,0.751594,-6.646896e-01,0.700204,0.699011
...,...,...,...,...,...,...,...,...,...
95,-0.999531,0.999808,-0.941649,0.077283,0.083094,4.335815,-1.293038e+01,0.288095,5.149160
96,0.969538,0.995283,0.243504,1.603972,0.265880,-0.018023,5.951827e-01,0.512599,-0.050871
97,-0.366343,0.998896,-0.991101,4.777041,0.245872,0.833548,-4.641661e-02,0.493358,0.342682
98,0.755199,0.999511,-0.490167,2.454086,0.549279,1.728212,2.674255e-01,0.722546,-3.345096


In [119]:
model_10

,x1(10),x2(10),x3(10),v1(10),v2(10),v3(10),y1(10),y2(10),y3(10)
0,0.963286,0.992305,-0.774704,5.899982,-0.461690,-0.269152,0.160244,0.667429,0.384272
1,-0.913723,0.999683,0.577276,9.024342,-0.251052,1.223504,-0.096785,0.498420,-1.650649
2,-0.762656,0.999997,0.225218,1.384979,-0.117757,3.477263,-0.480894,0.342761,-8.154377
3,0.466302,1.000000,0.984952,-2.632114,-0.374394,1.580380,-0.120975,0.604738,0.679232
4,0.397071,0.990962,-0.593515,0.007799,-0.043495,9.092895,32.083393,0.208522,-7.315947
...,...,...,...,...,...,...,...,...,...
95,0.548116,1.000000,-0.776058,1.064690,-0.183798,-1.144853,0.381141,0.427510,0.928193
96,0.468267,0.999823,0.134851,15.773478,0.120282,2.506338,0.020315,0.346399,4.997351
97,0.903188,0.999890,-0.857502,10.486523,0.449797,0.073321,0.081853,0.659380,-0.087423
98,0.927909,0.986300,-0.812237,-0.689711,0.116408,6.924005,-1.295958,0.340800,-9.020652


##1. Линейная форма функции пересчёта

In [120]:
def buildLinearRegretion(model1, model2, cols):
  vector_ones = [1] * elem
  coef = []

  linearFunction = (model1.copy())[cols[0:(len(cols) - 1)]]
  linearFunction[cols[len(cols) - 1]] = (model2.copy())[cols[len(cols) - 1]]
  linearFunction.insert(0, "optional_col", vector_ones)

  A_linear = np.array(linearFunction)
  #A_psev = BenIsrael(A_linear[:, 0:(len(A_linear[0]) - 1)])
  X_linear = solvingSLAE(A_linear[:, 0:(len(A_linear[0]) - 1)], np.matrix(A_linear[:, len(A_linear[0]) - 1]).T)

  estimate_linear = []
  variance_linear = []
  error_linear = []

  for i in range(len(linearFunction)):
    for j in range(0, len(X_linear)):
      if(j == 0):
        estimate_linear.append(X_linear[j,0])
      else:
        estimate_linear[i] += X_linear[j,0] * np.array(linearFunction)[i,j]

    variance_linear.append((1.0 / len(linearFunction)) * (np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])**2)
    #error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            #/ np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1]) * 100)
    error_linear.append((1.0 / len(linearFunction)) * abs((np.array(linearFunction)[i, len(np.array(linearFunction)[0]) - 1] - estimate_linear[i])
                            / max(np.array(linearFunction)[:, len(np.array(linearFunction)[0]) - 1])) * 100)

  linearFunction['regression'] = estimate_linear
  linearFunction['variance'] = variance_linear
  linearFunction['error, %'] = error_linear

  return X_linear, linearFunction

###Для функции $x_1[t+1]$

In [121]:
#x3, v1, v2
X1_linear, X1_linearTable = buildLinearRegretion(model_9, model_10, ['x3(9)', 'v1(9)', 'v2(9)', 'x1(10)'])
print(pd.DataFrame(X1_linear, columns=['X1'], index=['a1', 'c13', 'b11', 'b12']))
X1_linearTable

           X1
a1   0.070628
c13  0.019318
b11  0.000001
b12  0.113880


,optional_col,x3(9),v1(9),v2(9),x1(10),regression,variance,"error, %"
0,1,-0.594927,-0.124506,-0.498841,0.963286,0.002327,0.009234,0.961182
1,1,0.974466,0.647070,-0.293036,-0.913723,0.056083,0.009405,0.970031
2,1,-0.457796,-0.460634,-0.131483,-0.762656,0.046810,0.006552,0.809653
3,1,-0.331439,13.884109,0.054866,0.466302,0.070491,0.001567,0.395902
4,1,0.975114,-1.504231,0.512417,0.397071,0.147817,0.000621,0.249312
...,...,...,...,...,...,...,...,...
95,1,-0.941649,0.077283,0.083094,0.548116,0.061900,0.002364,0.486329
96,1,0.243504,1.603972,0.265880,0.468267,0.105612,0.001315,0.362738
97,1,-0.991101,4.777041,0.245872,0.903188,0.079488,0.006785,0.823891
98,1,-0.490167,2.454086,0.549279,0.927909,0.123714,0.006467,0.804381


In [122]:
print("E = " + str(sum(X1_linearTable['variance'])))
print("A = " + str(sum(X1_linearTable['error, %'])))

E = 0.4440660561928701
A = 59.24268172744737


###Для функции $x_2[t+1]$

In [123]:
#x2, x3, v1
X2_linear, X2_linearTable = buildLinearRegretion(model_9, model_10, ['x2(9)', 'x3(9)', 'v1(9)', 'x2(10)'])
print(pd.DataFrame(X2_linear, columns=['X2'], index=['a2', 'c22', 'c23', 'b21']))
X2_linearTable

               X2
a2   9.057212e-01
c22  9.096182e-02
c23 -7.401889e-04
b21  2.307755e-08


,optional_col,x2(9),x3(9),v1(9),x2(10),regression,variance,"error, %"
0,1,0.986387,-0.594927,-0.124506,0.992305,0.995885,1.281409e-07,0.003580
1,1,0.999324,0.974466,0.647070,0.999683,0.995900,1.431242e-07,0.003783
2,1,0.999932,-0.457796,-0.460634,0.999997,0.997016,8.890843e-08,0.002982
3,1,0.999925,-0.331439,13.884109,1.000000,0.996922,9.474776e-08,0.003078
4,1,0.999999,0.975114,-1.504231,0.990962,0.995961,2.498821e-07,0.004999
...,...,...,...,...,...,...,...,...
95,1,0.999808,-0.941649,0.077283,1.000000,0.997363,6.955166e-08,0.002637
96,1,0.995283,0.243504,1.603972,0.999823,0.996074,1.405970e-07,0.003750
97,1,0.998896,-0.991101,4.777041,0.999890,0.997316,6.621201e-08,0.002573
98,1,0.999511,-0.490167,2.454086,0.986300,0.997001,1.145305e-06,0.010702


In [124]:
print("E = " + str(sum(X2_linearTable['variance'])))
print("A = " + str(sum(X2_linearTable['error, %'])))

E = 3.189996398494474e-05
A = 0.4251117666078406


###Для функции $x_3[t+1]$

In [125]:
#x3, v1, v3
X3_linear, X3_linearTable = buildLinearRegretion(model_9, model_10, ['x3(9)', 'v1(9)', 'v3(9)', 'x3(10)'])
print(pd.DataFrame(X3_linear, columns=['X3'], index=['a3', 'c33', 'b31', 'b33']))
X3_linearTable

           X3
a3  -0.129058
c33  0.056665
b31 -0.000004
b33  0.025210


,optional_col,x3(9),v1(9),v3(9),x3(10),regression,variance,"error, %"
0,1,-0.594927,-0.124506,10.619837,-0.774704,0.104955,0.007738,0.879702
1,1,0.974466,0.647070,11.915311,0.577276,0.226540,0.001230,0.350753
2,1,-0.457796,-0.460634,20.778472,0.225218,0.368825,0.000206,0.143613
3,1,-0.331439,13.884109,15.828629,0.984952,0.251139,0.005385,0.733848
4,1,0.975114,-1.504231,0.751594,-0.593515,-0.054850,0.002902,0.538691
...,...,...,...,...,...,...,...,...
95,1,-0.941649,0.077283,4.335815,-0.776058,-0.073112,0.004941,0.702980
96,1,0.243504,1.603972,-0.018023,0.134851,-0.115721,0.000628,0.250585
97,1,-0.991101,4.777041,0.833548,-0.857502,-0.164225,0.004806,0.693311
98,1,-0.490167,2.454086,1.728212,-0.812237,-0.113276,0.004885,0.698996


In [126]:
print("E = " + str(sum(X3_linearTable['variance'])))
print("A = " + str(sum(X3_linearTable['error, %'])))

E = 0.5028742811211798
A = 63.319839160573835


###Для функции $y_1[t]$

In [127]:
#x1, x2, v1
Y1_linear, Y1_linearTable = buildLinearRegretion(model_9, model_9, ['x1(9)', 'x2(9)', 'v1(9)', 'y1(9)'])
print(pd.DataFrame(Y1_linear, columns=['Y1'], index=['a1', 'c11', 'c12', 'b11']))
Y1_linearTable

               Y1
a1   2.361247e+01
c11  5.176866e-01
c12 -2.383630e+01
b11  7.126251e-07


,optional_col,x1(9),x2(9),v1(9),y1(9),regression,variance,"error, %"
0,1,-0.256393,0.986387,-0.124506,1.042722e+00,-0.032074,0.011552,0.875551
1,1,0.377749,0.999324,0.647070,3.588017e-01,-0.012153,0.001376,0.302187
2,1,0.893641,0.999932,-0.460634,-1.833957e+00,0.240427,0.043031,1.689836
3,1,0.990310,0.999925,13.884109,7.098042e-02,0.290637,0.000482,0.178937
4,1,0.999898,0.999999,-1.504231,-6.646896e-01,0.293832,0.009188,0.780831
...,...,...,...,...,...,...,...,...
95,1,-0.999531,0.999808,0.077283,-1.293038e+01,-0.736695,1.486859,9.933225
96,1,0.969538,0.995283,1.603972,5.951827e-01,0.390532,0.000419,0.166712
97,1,-0.366343,0.998896,4.777041,-4.641661e-02,-0.387171,0.001161,0.277585
98,1,0.755199,0.999511,2.454086,2.674255e-01,0.178782,0.000079,0.072210


In [128]:
print("E = " + str(sum(Y1_linearTable['variance'])))
print("A = " + str(sum(Y1_linearTable['error, %'])))

E = 1.8392435931719218
A = 43.83799607619724


###Для функции $y_2[t]$

In [129]:
#v3
Y2_linear, Y2_linearTable = buildLinearRegretion(model_9, model_9, ['v3(9)', 'y2(9)'])
print(pd.DataFrame(Y2_linear, columns=['Y2'], index=['a2', 'b23']))
Y2_linearTable

           Y2
a2   0.496569
b23  0.003370


,optional_col,v3(9),y2(9),regression,variance,"error, %"
0,1,10.619837,0.691670,0.532363,2.537879e-04,0.212892
1,1,11.915311,0.537457,0.536729,5.296972e-09,0.000973
2,1,20.778472,0.362083,0.566603,4.182820e-04,0.273311
3,1,15.828629,0.234177,0.549919,9.969345e-04,0.421945
4,1,0.751594,0.700204,0.499102,4.044187e-04,0.268744
...,...,...,...,...,...,...
95,1,4.335815,0.288095,0.511183,4.976806e-04,0.298125
96,1,-0.018023,0.512599,0.496508,2.589205e-06,0.021503
97,1,0.833548,0.493358,0.499378,3.624319e-07,0.008045
98,1,1.728212,0.722546,0.502394,4.846689e-04,0.294202


In [130]:
print("E = " + str(sum(Y2_linearTable['variance'])))
print("A = " + str(sum(Y2_linearTable['error, %'])))

E = 0.03140447026510537
A = 19.98165270732317


###Для функции $y_3[t]$

In [131]:
#v1, v3
Y3_linear, Y3_linearTable = buildLinearRegretion(model_9, model_9, ['v1(9)', 'v3(9)', 'y3(9)'])
print(pd.DataFrame(Y3_linear, columns=['Y3'], index=['a3', 'b31', 'b33']))
Y3_linearTable

           Y3
a3  -1.642129
b31 -0.000010
b33 -0.281651


,optional_col,v1(9),v3(9),y3(9),regression,variance,"error, %"
0,1,-0.124506,10.619837,5.501528,-4.633212,1.027129,0.067466
1,1,0.647070,11.915311,4.189059,-4.998090,0.844037,0.061158
2,1,-0.460634,20.778472,-51.896833,-7.494394,19.715766,0.295582
3,1,13.884109,15.828629,-67.729553,-6.100408,37.981515,0.410258
4,1,-1.504231,0.751594,0.699011,-1.853801,0.065169,0.016994
...,...,...,...,...,...,...,...
95,1,0.077283,4.335815,5.149160,-2.863315,0.641998,0.053338
96,1,1.603972,-0.018023,-0.050871,-1.637068,0.025160,0.010559
97,1,4.777041,0.833548,0.342682,-1.876945,0.049267,0.014776
98,1,2.454086,1.728212,-3.345096,-2.128905,0.014791,0.008096


In [132]:
print("E = " + str(sum(Y3_linearTable['variance'])))
print("A = " + str(sum(Y3_linearTable['error, %'])))

E = 527.4974694962626
A = 6.121956449276471


##2. Квадратичная форма функции пересчёта